# CAISO Demand Scraper

This notebook contains code for downloading and aggregating California electrical demand data from the California Independent System Operator (CAISO) website. The data come in daily time-series files at 5 minute intervals with columns for hour-ahead forecast, total demand, and net demand (demand minus renewables).

To collect new data you will need to do the following steps:
1. <a href='#date_range'>Set the date range you want new data over</a>
2. <a href='#downloads_path'>Specify a path to your downloads folder for the aggregating function</a> 
3. <a href='#download_data'>Download data</a>
4. <a href='#aggregate_data'>Aggregate data</a>

### Imports

In [1]:
import pandas as pd
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

### Functions

#### Demand Scraper

In [2]:
def download_demand(date_list):
    # Instantiate chrome environment
    driver = webdriver.Chrome(ChromeDriverManager().install())

    # Go to this caiso page
    driver.get('http://caiso.com/TodaysOutlook/Pages/default.aspx')
    
    date_xpath = "//input[@type = 'text' and @class ='form-control date demand-date']"
    
    rest = 30
    count = 0
    
    for date in dates:
        
        # Go to the date toggle box, clear the input
        driver.find_element_by_xpath(date_xpath).clear()

        # Type in the date of your choice!
        driver.find_element_by_xpath(date_xpath).send_keys(date)

        # This clicks enter
        driver.find_element_by_xpath(date_xpath).send_keys(u'\ue007') 
        
        # This clicks the renewable trend's data button
        driver.find_element_by_xpath("//button[@id='dropdownMenu1']").click()

        # This clicks the supply trend's download csv
        driver.find_element_by_xpath("//a[@id='downloadDemandCSV']").click()

        count += 1 
        
        # sleep function:  rest every 30 files to not get booted off of site
        if count > rest:
            count = 0
            sleep(120)
    
    # sleep for 1 second to allow final download, then close browser
    sleep(1)
    driver.close()

#### Demand aggregation

In [3]:
def agg_dem(file):
    times = []
    forecast = []
    demand = []
    net_demand = []

    for t in file.columns[1:-2]:
        times.append(file.columns[0][-10:] + 'T' + t)
        forecast.append(file[t][0])
        demand.append(file[t][1])
        net_demand.append(file[t][2])
            
    nu_df = pd.DataFrame({'date':times,'forecast':forecast,'demand':demand,'net_demand':net_demand})
    nu_df['date'] = pd.to_datetime(nu_df['date'])  
    return nu_df   

### Set date range and downloads path

<a id='date_range'></a>

<a id='downloads_path'></a>

In [4]:
##### EDIT THIS CELL ######

# Set starting and ending dates
early_date = '06/01/2020'
later_date = '06/03/2020'

# Set path to downloads folder (this sample is the default for Mac OS)
downloads_path = '~/Downloads/'

In [5]:
# Define list of dates between specified start and end
dates = pd.date_range(early_date, later_date).tolist()
dates = [date.strftime("%m/%d/%Y") for date in dates]

# reformat date range to match downloaded file format
files = [downloads_path+'CAISO-demand-'+day[-4:]+day[:2]+day[3:5]+'.csv' for day in dates]

### Download data

<a id='download_data'></a>

In [6]:
download_demand(dates)

[WDM] - Cache is valid for [04/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 mac64] driver in cache 
[WDM] - Driver found in cache [/Users/Sam/.wdm/drivers/chromedriver/83.0.4103.39/mac64/chromedriver]


### Aggregate data to pandas dataframe, export to CSV

<a id='aggregate_data'></a>

In [7]:
# allocate space for aggregate data
demand_series = []

for file in files:
    nu_df = agg_dem(pd.read_csv(file))
    demand_series.append(nu_df)
    
demand = pd.concat(demand_series)

demand.to_csv('../datasets/sample_demand_series.csv',index=False)